In [2]:
from os import listdir
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import jieba.analyse
import jieba
from joblib import Parallel, delayed

In [2]:
folder_dir = "C:/Users/tsunh/Desktop/Schoolwork/NLPlab/sr_chat_data/sr_chat_data"
file_path = [ folder_dir +"/"+ path for path in listdir(folder_dir) ]
file_name = listdir(folder_dir)
print file_path[:3]
print file_name[:3]

['C:/Users/tsunh/Desktop/Schoolwork/NLPlab/sr_chat_data/sr_chat_data/chat_0003e590ac9f414a51c1245bc02f17eb.csv', 'C:/Users/tsunh/Desktop/Schoolwork/NLPlab/sr_chat_data/sr_chat_data/chat_000403321513e40db8000af807ec7f96.csv', 'C:/Users/tsunh/Desktop/Schoolwork/NLPlab/sr_chat_data/sr_chat_data/chat_00067296c7b74fec845c2368964f430c.csv']
['chat_0003e590ac9f414a51c1245bc02f17eb.csv', 'chat_000403321513e40db8000af807ec7f96.csv', 'chat_00067296c7b74fec845c2368964f430c.csv']


In [3]:
chat_frame = pd.read_csv(file_path[0])
chat_frame.head(20)

,receiver,sender,text,time,type,sender_gender
0,0003e590ac9f414a51c1245bc02f17eb,0b39d793a34f41903319898f24737bc8,Hello..妳好...I am Thomas...希望有機會與妳聊聊...更希望彼此是彼此...,1464245603,7,m
1,0003e590ac9f414a51c1245bc02f17eb,0b39d793a34f41903319898f24737bc8,希望有機會與Sake聊聊,1464245617,0,m
2,0b39d793a34f41903319898f24737bc8,0003e590ac9f414a51c1245bc02f17eb,嗯嗯 我在上班 下班晚點可以聊聊,1464245695,0,f
3,0b39d793a34f41903319898f24737bc8,0003e590ac9f414a51c1245bc02f17eb,^_^,1464245696,0,f
4,0003e590ac9f414a51c1245bc02f17eb,0b39d793a34f41903319898f24737bc8,好的,1464245716,0,m
5,0b39d793a34f41903319898f24737bc8,0003e590ac9f414a51c1245bc02f17eb,哈 謝謝,1464245746,0,f
6,0003e590ac9f414a51c1245bc02f17eb,0b39d793a34f41903319898f24737bc8,😊,1464245761,0,m
7,0003e590ac9f414a51c1245bc02f17eb,0b39d793a34f41903319898f24737bc8,妳先忙工作,1464245771,0,m
8,0b39d793a34f41903319898f24737bc8,0003e590ac9f414a51c1245bc02f17eb,嗯嗯好 你也是,1464245872,0,f
9,0003e590ac9f414a51c1245bc02f17eb,0b39d793a34f41903319898f24737bc8,下班了嗎？,1464261204,0,m


In [4]:
print chat_frame.loc[6,"text"]

😊


### keep emoji (extract emoji element previously, then cut the words, finally append the emoji back)

In [5]:
#build emoji dictionary
import requests
from bs4 import BeautifulSoup
emoji_list = []
url = "https://apps.timwhitlock.info/emoji/tables/unicode"
r = requests.get(url).content
soup = BeautifulSoup(r)
for ele in soup.find_all("td", class_="code"):
    emo_ele = ele.text
    if emo_ele.startswith("\\"):
        emoji_list.append(emo_ele.decode("utf8").decode('string_escape')) #remove redundant backslash for latter matching

c:\python27\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [90]:
chat_frame.loc[6,"text"] in emoji_list

True

In [91]:
emoji_list[1]

'\xf0\x9f\x98\x82'

#### Write emoji character into jieba dictionary

In [6]:
import re
text = chat_frame.loc[6,"text"]
print text
def emoji_extractor(text):
    text = text.decode("utf8")
    try:
        # UCS-4
        highpoints = re.compile(u'[\U00010000-\U0010ffff]')
    except re.error:
        # UCS-2
        highpoints = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')
    return highpoints.sub("Emoji ", text)
print emoji_extractor(text)

😊
Emoji 


### 斷詞

In [7]:
#customize dictionary
jieba.set_dictionary('C:\\Python27\\Lib\\site-packages\\jieba\\dict.txt.big.txt')
jieba.analyse.set_stop_words("C:\\Python27\\Lib\\site-packages\\jieba\\stop_words.txt")
jieba.analyse.set_idf_path("C:\\Python27\\Lib\\site-packages\\jieba\\idf.txt.big.txt")

In [8]:
for i in range(chat_frame.shape[0]):
    text = chat_frame.loc[i,"text"]
    text_ele = emoji_extractor(text)
    chat_frame.loc[i,"seg_text"] = " ".join(jieba.cut(text_ele))

Building prefix dict from C:\Python27\Lib\site-packages\jieba\dict.txt.big.txt ...
Loading model from cache c:\users\tsunh\appdata\local\temp\jieba.u7ecb02fee068d7d39e5500de2047abc1.cache
Loading model cost 0.668 seconds.
Prefix dict has been built succesfully.


In [9]:
print chat_frame["seg_text"][6]

Emoji  


### batch word segment and save it

In [10]:
seg_folder_dir = "C:/Users/tsunh/Desktop/Schoolwork/NLPlab/sr_chat_data/sr_chat_data_seg/"
seg_path = [ seg_folder_dir +"/"+ path for path in listdir(seg_folder_dir) ]
seg_name = listdir(seg_folder_dir)

folder_dir = "C:/Users/tsunh/Desktop/Schoolwork/NLPlab/sr_chat_data/sr_chat_data"
file_path = [ folder_dir +"/"+ path for path in listdir(folder_dir) if path not in seg_name ]
file_name = [i for i in listdir(folder_dir) if i not in seg_name]
print len(file_path)

19632


In [12]:
print len(file_name)

19632


In [13]:
for path, name in zip(file_path,file_name):
    chat_frame = pd.read_csv(path)
    for row in range(chat_frame.shape[0]):
        text = chat_frame.loc[row,"text"]
        try:
            text_ele = emoji_extractor(text)
            chat_frame.loc[row,"seg_text"] = " ".join(jieba.cut(text_ele))
        except:
            chat_frame.loc[row,"seg_text"] = " "
    chat_frame.to_csv("C:/Users/tsunh/Desktop/Schoolwork/NLPlab/sr_chat_data/sr_chat_data_seg/"+name,
                      index = False,
                     encoding="utf8")

## Word2Vec 

In [2]:
#load chinese word2vec model
word2vec = pd.read_csv("c:/Users/tsunh/Desktop/Schoolwork/NLPlab/wiki_wordvector_utf8.csv",index_col = 0)

In [3]:
word2vec.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100
gobies,-0.640565,0.033635,0.413354,0.001317,0.401215,-0.072820,-0.295357,1.005226,-0.186935,0.215594,...,0.196001,-0.294534,-0.734109,0.079843,0.706463,-0.475461,-0.280217,-0.067575,-0.103508,0.071767
Gorsuch,-0.267785,0.249702,-0.087891,-0.568892,0.344263,-0.252007,-0.730565,0.439203,0.282937,-0.585860,...,0.949666,-0.311173,-0.281239,0.430901,0.382697,-0.381894,0.151197,0.962285,-0.618023,0.688148
由奧恰,0.010707,0.175031,-0.574771,0.530207,0.146367,-0.002879,-0.008207,-0.573806,0.449668,0.985068,...,0.393548,-0.425076,0.473486,-0.124235,0.068624,0.631609,0.641269,-0.575997,-0.397369,-0.693963
朝義著,-0.425897,-0.019429,-0.340932,0.088261,-0.585137,-0.280552,0.031831,-0.265396,-0.211374,-0.132702,...,-0.059611,-0.648249,-0.707557,0.224414,0.688562,0.134534,-0.808125,0.879801,-0.356066,0.401297
calabai,0.295467,-0.605804,0.100886,0.134296,-0.275335,-0.487929,-0.092787,-0.483964,-0.285548,-0.504646,...,0.853770,0.038695,0.309583,0.751271,-0.084114,-0.082777,0.148701,0.175883,0.858911,-0.589927


In [4]:
word2vec.shape

(268569, 100)

In [5]:
def vector_sim(x,y):
    return cosine_similarity(np.array(x).reshape(1,-1),np.array(y).reshape(1,-1))
print vector_sim(word2vec.loc["新竹"],word2vec.loc["桃園"])
print vector_sim(word2vec.loc["雲林"],word2vec.loc["桃園"])
print vector_sim(word2vec.loc["雲林"],word2vec.loc["嘉義"])

[[ 0.69812921]]
[[ 0.52460531]]
[[ 0.67427972]]


### document represented by word2vec average (median) 

In [7]:
seg_folder_dir = "C:/Users/tsunh/Desktop/Schoolwork/NLPlab/sr_chat_data/sr_chat_data_seg/"
seg_path = [ seg_folder_dir +"/"+ path for path in listdir(seg_folder_dir) ]
seg_name = listdir(seg_folder_dir)

In [8]:
test_df = pd.read_csv(seg_path[1])
test_df.head()

,receiver,sender,text,time,type,sender_gender,seg_text
0,000403321513e40db8000af807ec7f96,42132af3ebc9a36017752a0d67d5e09d,嗨！ 妳好阿,1444407467,0,m,嗨 ！ 妳 好 阿
1,42132af3ebc9a36017752a0d67d5e09d,000403321513e40db8000af807ec7f96,嗨~,1444437791,0,f,嗨 ~
2,000403321513e40db8000af807ec7f96,42132af3ebc9a36017752a0d67d5e09d,休假沒有出去玩,1444448624,0,m,休假 沒有 出去玩
3,000403321513e40db8000af807ec7f96,42132af3ebc9a36017752a0d67d5e09d,妳因該很少來甜甜圈這裡吧,1444448698,0,m,妳 因該 很少 來 甜甜 圈 這裡 吧
4,42132af3ebc9a36017752a0d67d5e09d,000403321513e40db8000af807ec7f96,我昨天晚上才下載看看的！哈哈,1444455165,0,f,我 昨天晚上 才 下載 看看 的 ！ 哈哈


In [ ]:
def create_d2v_file(path,name):
    column_name = ["v"+str(i) for i in range(1,101)]
    df = pd.read_csv(path)
    vec_frame = []
    for row in range(df.shape[0]):
        word2vec2doc = []
        
        for word in df.loc[row,"seg_text"].split(" "):
                try:
                    word2vec2doc.append(np.array(word2vec.loc[word]))
                except:
                    word2vec2doc.append(np.zeros(100))
                    
        vec_frame.append(np.mean(np.array(word2vec2doc),axis = 0))
    frame_with_DocVec = pd.concat([df, pd.DataFrame(np.array(vec_frame),columns=column_name)], axis=1)
    frame_with_DocVec.to_csv("C:/Users/tsunh/Desktop/Schoolwork/NLPlab/sr_chat_data/sr_chat_data_DV/"+name,
                          index = False,
                         encoding="utf8")
    
# for path, name in zip(seg_path,seg_name):
#     create_d2v_file(path,name)
Parallel(n_jobs=2)(delayed(create_d2v_file)(path,name) for path, name in zip(seg_path,seg_name))

In [61]:
word2vec2doc = []
for word in test_df.loc[0,"seg_text"].split(" "):
    print word
    try:
        word2vec2doc.append(np.array(word2vec.loc[word]))
    except:
        word2vec2doc.append(np.zeros(100))

np.mean(np.array(word2vec2doc),axis = 0)

嗨
！


妳
好
阿


array([-0.01265901, -0.28645023, -0.12414637,  0.15532378,  0.06675993,
        0.21572613,  0.01586927,  0.01928163, -0.16977498,  0.49181543,
       -0.18359625,  0.21302659, -0.21054676, -0.10143435,  0.36809709,
       -0.11277422,  0.07332392, -0.34659433,  0.25986605,  0.06379166,
        0.50997516, -0.12019864, -0.23410239,  0.22958985, -0.2011734 ,
        0.02703978,  0.36428104, -0.19721388,  0.14094882, -0.35433115,
        0.74377856,  0.28540112, -0.19547316,  0.21906016,  0.06786173,
       -0.03714963, -0.26161875,  0.20787675, -0.13378314,  0.43746244,
        0.02895689,  0.07127717,  0.49596735,  0.06672881, -0.37759267,
        0.1423058 , -0.15724532, -0.42832666, -0.48626658, -0.25020022,
       -0.31862739, -0.17347516, -0.00472732,  0.38206937,  0.09703728,
        0.20031133, -0.00076469, -0.71390074,  0.00878887, -0.15457282,
       -0.18275521,  0.05956632,  0.14006899,  0.15386101,  0.0885594 ,
        0.11892621,  0.14480148,  0.1052328 , -0.21038257, -0.16

### The above method is too slow, try gensim doc2vec

In [1]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

In [15]:
#try to load all the chat content
seg_folder_dir = "C:/Users/tsunh/Desktop/Schoolwork/NLPlab/sr_chat_data/sr_chat_data_seg/"
seg_path = [ seg_folder_dir +"/"+ path for path in listdir(seg_folder_dir) ]
seg_name = listdir(seg_folder_dir)
all_chat = pd.DataFrame()

i = 0
for path, name in zip(seg_path, seg_name):
    all_chat = all_chat.append(pd.read_csv(path),ignore_index=True)
    i+=1
    if i%1000==0:
        print i

1000
2000
3000


KeyboardInterrupt: 

In [16]:
all_chat

,receiver,seg_text,sender,sender_gender,text,time,type,﻿receiver
0,0003e590ac9f414a51c1245bc02f17eb,Hello .. 妳 好 ... I am Thomas ... 希望 有 機會 與...,0b39d793a34f41903319898f24737bc8,m,Hello..妳好...I am Thomas...希望有機會與妳聊聊...更希望彼此是彼此...,1464245603,7,NaN
1,0003e590ac9f414a51c1245bc02f17eb,希望 有 機會 與 Sake 聊聊,0b39d793a34f41903319898f24737bc8,m,希望有機會與Sake聊聊,1464245617,0,NaN
2,0b39d793a34f41903319898f24737bc8,嗯 嗯 我 在 上班 下班 晚點 可以 聊聊,0003e590ac9f414a51c1245bc02f17eb,f,嗯嗯 我在上班 下班晚點可以聊聊,1464245695,0,NaN
3,0b39d793a34f41903319898f24737bc8,^ _ ^,0003e590ac9f414a51c1245bc02f17eb,f,^_^,1464245696,0,NaN
4,0003e590ac9f414a51c1245bc02f17eb,好 的,0b39d793a34f41903319898f24737bc8,m,好的,1464245716,0,NaN
5,0b39d793a34f41903319898f24737bc8,哈 謝謝,0003e590ac9f414a51c1245bc02f17eb,f,哈 謝謝,1464245746,0,NaN
6,0003e590ac9f414a51c1245bc02f17eb,Emoji,0b39d793a34f41903319898f24737bc8,m,😊,1464245761,0,NaN
7,0003e590ac9f414a51c1245bc02f17eb,妳 先忙 工作,0b39d793a34f41903319898f24737bc8,m,妳先忙工作,1464245771,0,NaN
8,0b39d793a34f41903319898f24737bc8,嗯 嗯 好 你 也 是,0003e590ac9f414a51c1245bc02f17eb,f,嗯嗯好 你也是,1464245872,0,NaN
9,0003e590ac9f414a51c1245bc02f17eb,下班 了 嗎 ？,0b39d793a34f41903319898f24737bc8,m,下班了嗎？,1464261204,0,NaN
